# Part 2: Feature Engineering & Training Base Construction

This notebook builds rich, **leak‑aware** features at the user, product, and user×product (UXP) levels and assembles the **training base (`df.pkl`)** for the Instacart Reorder Prediction model.

## Files Used (inputs)

* `prior_data.pkl` – prior orders with user, product, time, aisle/department info
* `orders.pkl` – order metadata (used to extract the train order)
* `order_products_train.pkl` – labeled products for each user’s **train** order

## Outputs

* `user_features.pkl` – user‑level feature table
* `product_features.pkl` – product/aisle/department feature table
* `up.pkl` – user×product interaction features
* `df.pkl` – final modeling dataset after merging labels & features (no nulls)

## Workflow Summary

### 1) User‑Level Features (`user_features`)

Aggregated from `prior_data` only.

* **Order & product volume**
  * `user_total_orders` (max order number in prior)
  * `user_total_products`, `user_distinct_products`
  * `user_avg_basket_size` = count / nunique (proxy)
* **Cadence & gaps**
  * `user_avg_days_between_orders`, `user_min/max/avg_order_gap`, `user_median_days_between_orders`
* **Timing preferences**
  * `user_avg_order_hour`, `user_most_common_dow`
* **Reordering behavior**
  * `user_reorder_ratio` (on orders >1 only)
* **Weekend & temporal variability**
  * `user_weekend_order_ratio` (Sun/Mon=0/6 treated; weekend = {0,6})
  * `user_most_common_order_hour`, `user_hour_std`
  * `user_hour_entropy` (Shannon entropy of order hours; higher = more spread)

> Saved to `user_features.pkl`.

### 2) Product‑Level Features (`product_features`)

From `prior_data`, augmented with aisle/department rollups.

* **Core product stats:**
  `prod_total_purchases`, `prod_reorder_count`, `prod_reorder_rate`,
  `prod_avg_cart_position`, `prod_median_cart_position`,
  `prod_first_cart_order_avg` (% of times added first), `prod_distinct_users`
* **Category context:**
  `aisle_total_orders`, `aisle_reorder_rate`, `aisle_distinct_products`,
  `dept_total_orders`, `dept_reorder_rate`, `dept_reorder_share`

> Saved to `product_features.pkl`.

### 3) User × Product Interaction Features (`up`)

Built from `prior_data` at the (user\_id, product\_id) grain.

* **Frequency & recency markers**
  `up_order_count`, `up_first_order_number`, `up_last_order_number`,
  `up_orders_since_last` (= user\_total\_orders − last\_order\_number)
* **Cart behavior**
  `up_avg_cart_position`
* **Reordering flags & rates**
  `up_total_reordered`, `up_reorder_flag` (ever reordered =1),
  `up_reorder_ratio` (= order\_count / user\_total\_orders),
  `up_order_rate_since_first` (= order\_count / (user\_total\_orders − first\_order\_number + 1))
* **Time since last**
  `up_days_since_last_order` via cumulative sum of `days_since_prior_order`

> Saved to `up.pkl`.

### 4) Merging Target Label and Engineered Features for Modeling

This stage constructs the final training dataset (`df.pkl`) by combining labels from the train order with the engineered feature sets.

### Workflow

1. **Build `train_orders`**
   - Merge the orders table with order_products_train to attach the reordered label from order_products_train to each corresponding order in orders.
   - Kept only rows where `eval_set = 'train'`.

2. **Filter to relevant users**
   - From train_orders, collect the unique user IDs and use them to filter the UXP (user×product) feature table. This keeps only rows for train-set users and removes any users that exist only in the test set, ensuring all remaining rows have a corresponding label.

3. **Outer merge with User×Product (UXP) features**
   - Performed an outer join between `train_orders` and the `up` table.
   - This approach keeps:
         - Products reordered in the train order (positives)
         - Products bought previously but not reordered in the train order **(assumed negatives)**
         - Products purchased for the first time in the train order (new items with no prior history)
   - An outer join ensures we retain all relevant cases instead of losing rows with no match on one side. (An inner join would drop the assumed negatives and new items, leaving out important training examples.)

4. **Impute missing order details**
   - After the outer join, some rows had missing values for:
     - `order_number`
     - `order_dow` (day of week)
     - `order_hour_of_day`
     - `days_since_prior_order`
   - Missing values were filled using **user-level averages**, preserving each user’s ordering habits rather than inserting a generic value.

5. **Handle labels (`reordered` column)**
   - **Negative-drop:** Removed rows where `reordered = 0` **and** the product was explicitly in the train order list (explicit negatives). These rows are extremely numerous, heavily skew the dataset toward zeros, and add little unique signal.
   - **Filled NaNs with 0:** Rows that never appeared in the train order had `NaN` in `reordered`. These are *assumed negatives* — products the user has purchased before but did not reorder in the train order. We filled these with `0` to provide meaningful counterexamples.
   - **Why treated differently?**
     - **Explicit negatives:** Too many, cause imbalance, minimal extra insight. --> Removed
     - **Assumed negatives:** Smaller, tied to real purchase history, and useful for the model to learn “no reorder” cases. --> Kept

6. **Merge in product and user features**
   - Merged `product_features` (popularity, reorder rates, aisle/department context) on `product_id`.
   - Merged `user_features` (shopping frequency, basket size, timing patterns) on `user_id`.

7. **Final validation**
   - Verified **no missing values** across all columns.
   - Ensured all features are numeric except ID fields, making the dataset model-ready.

### Result
A fully numeric, multi-million-row dataset (~8.47M rows after label handling) with:
- Detailed user×product interaction features
- Rich product statistics
- Comprehensive user-level behavior profiles

> Saved as `df.pkl`.

## Outcome

* Three feature stores (`user_features`, `product_features`, `up`) + merged **modeling base (`df.pkl`)** with **no missing values**.
* Comprehensive behavioral, temporal, and category context signals assembled for the reorder classifier.

## Next Steps

1. **Modeling:** Train baseline (e.g., Logistic/XGBoost)
2. **Threshold tuning:** Optimize F1
3. **Explainability:** Run SHAP to validate drivers and detect any lingering leakage patterns.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from scipy.stats import entropy
import sys

In [2]:
prior_data = pd.read_pickle('prior_data.pkl')

In [3]:
orders = pd.read_pickle('orders.pkl')

In [4]:
order_products_train = pd.read_pickle('order_products_train.pkl')

In [5]:
prior_data.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,soft drinks,beverages
1,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,soy lactosefree,dairy eggs
2,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,popcorn jerky,snacks
3,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,popcorn jerky,snacks
4,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,paper goods,household


Feature Engineering

User-level Features

In [8]:
# Group by user_id and compute basic user-level aggregates
user_features = prior_data.groupby('user_id').agg(
    user_total_orders=('order_number', 'max'),  # Total number of orders placed by the user
    user_total_products=('product_id', 'count'),  # Total number of products purchased (including repeats)
    user_distinct_products=('product_id', 'nunique'),  # Number of unique products purchased
    user_avg_basket_size=('product_id', lambda x: x.count() / x.nunique()),  # Proxy for avg basket size
    user_avg_days_between_orders=('days_since_prior_order', 'mean'),  # Avg days between orders
    user_avg_order_hour=('order_hour_of_day', 'mean'),  # Avg hour the user places orders
    user_most_common_dow=('order_dow', lambda x: x.mode()[0])  # Most frequent day of the week user orders
).reset_index()

# Reorder ratio (excluding first order)

reordered_by_user = prior_data[prior_data['order_number'] > 1].groupby('user_id')['reordered'].sum()  # # reorders
total_by_user = prior_data[prior_data['order_number'] > 1].groupby('user_id')['product_id'].count()  # # items ordered
user_features['user_reorder_ratio'] = (reordered_by_user / total_by_user).fillna(0).values  # Ratio

# Add order gap stats: median, min, max

user_order_gap_stats = prior_data.groupby('user_id')['days_since_prior_order'].agg([
    ('user_min_order_gap', 'min'),           # Min days between orders
    ('user_max_order_gap', 'max'),           # Max days between orders
    ('user_avg_order_gap', 'mean'),          # Avg days between orders
    ('user_median_days_between_orders', 'median')  # Median days between orders
]).fillna(0).reset_index()

# Merge gap stats with user_features
user_features = user_features.merge(user_order_gap_stats, on='user_id', how='left')

# Preview final user-level feature table
display(user_features.head())

,user_id,user_total_orders,user_total_products,user_distinct_products,user_avg_basket_size,user_avg_days_between_orders,user_avg_order_hour,user_most_common_dow,user_reorder_ratio,user_min_order_gap,user_max_order_gap,user_avg_order_gap,user_median_days_between_orders
0,1,10,59,18,3.277778,20.259259,10.542373,4,0.759259,0.0,30.0,20.259259,20.5
1,2,14,195,102,1.911765,15.967033,10.441026,2,0.510989,3.0,30.0,15.967033,13.0
2,3,12,88,33,2.666667,11.487179,16.352273,0,0.705128,7.0,21.0,11.487179,10.0
3,4,5,18,17,1.058824,15.357143,13.111111,4,0.071429,0.0,21.0,15.357143,20.0
4,5,4,37,23,1.608696,14.500000,15.729730,3,0.538462,10.0,19.0,14.500000,11.0


In [9]:
# Identify weekend orders
prior_data['is_weekend'] = prior_data['order_dow'].isin([0, 6])

# Entropy function
def calc_entropy(x):
    counts = x.value_counts()
    return entropy(counts, base=2)

# User-level datetime stats
user_dt = prior_data.groupby('user_id').agg(
    user_weekend_order_ratio=('is_weekend', 'mean'),
    user_most_common_order_hour=('order_hour_of_day', lambda x: x.mode()[0]),
    user_hour_std=('order_hour_of_day', 'std')
).reset_index()

# Add entropy
user_entropy = prior_data.groupby('user_id')['order_hour_of_day'].apply(calc_entropy).reset_index()
user_entropy.columns = ['user_id', 'user_hour_entropy']

# Final user-level datetime feature table
user_dt = user_dt.merge(user_entropy, on='user_id', how='left')

display(user_dt.head())

,user_id,user_weekend_order_ratio,user_most_common_order_hour,user_hour_std,user_hour_entropy
0,1,0.000000,7,3.500355,2.659844
1,2,0.000000,9,1.649854,2.097470
2,3,0.522727,16,1.454599,2.477860
3,4,0.222222,15,1.745208,1.571542
4,5,0.243243,18,2.588958,1.531887


In [10]:
user_features = user_features.merge(user_dt, on='user_id', how='left')
display(user_features.head())

,user_id,user_total_orders,user_total_products,user_distinct_products,user_avg_basket_size,user_avg_days_between_orders,user_avg_order_hour,user_most_common_dow,user_reorder_ratio,user_min_order_gap,user_max_order_gap,user_avg_order_gap,user_median_days_between_orders,user_weekend_order_ratio,user_most_common_order_hour,user_hour_std,user_hour_entropy
0,1,10,59,18,3.277778,20.259259,10.542373,4,0.759259,0.0,30.0,20.259259,20.5,0.000000,7,3.500355,2.659844
1,2,14,195,102,1.911765,15.967033,10.441026,2,0.510989,3.0,30.0,15.967033,13.0,0.000000,9,1.649854,2.097470
2,3,12,88,33,2.666667,11.487179,16.352273,0,0.705128,7.0,21.0,11.487179,10.0,0.522727,16,1.454599,2.477860
3,4,5,18,17,1.058824,15.357143,13.111111,4,0.071429,0.0,21.0,15.357143,20.0,0.222222,15,1.745208,1.571542
4,5,4,37,23,1.608696,14.500000,15.729730,3,0.538462,10.0,19.0,14.500000,11.0,0.243243,18,2.588958,1.531887


In [11]:
user_features.to_pickle('user_features.pkl')

Product-Level Features

In [13]:
# Product-Level Features from prior_data
product_features = prior_data.groupby('product_id').agg(
    prod_total_purchases=('reordered', 'count'),                     # Total purchases
    prod_reorder_count=('reordered', 'sum'),                         # Total reorders
    prod_reorder_rate=('reordered', 'mean'),                         # Reorder ratio
    prod_avg_cart_position=('add_to_cart_order', 'mean'),            # Avg position in cart
    prod_distinct_users=('user_id', 'nunique'),                      # Unique users
    prod_first_cart_order_avg=('add_to_cart_order', lambda x: (x == 1).mean()),  # % first in cart
    prod_median_cart_position=('add_to_cart_order', 'median')        # Median position in cart
).reset_index()

# Reorder Probability: users who bought product > once
user_product = prior_data.groupby(['product_id', 'user_id']).agg(
    prod_user_orders=('order_id', 'count'),
    prod_user_reorders=('reordered', 'sum')
).reset_index()

reordered_users = user_product[user_product['prod_user_orders'] > 1].groupby('product_id')['prod_user_reorders'].sum()
users_bought_more_than_once = user_product[user_product['prod_user_orders'] > 1].groupby('product_id')['user_id'].count()

# Aisle-Level Features
aisle_features = prior_data.groupby('aisle').agg(
    aisle_total_orders=('reordered', 'count'),
    aisle_reorder_rate=('reordered', 'mean'),
    aisle_distinct_products=('product_id', 'nunique')
).reset_index()

# Department-Level Features
dept_features = prior_data.groupby('department').agg(
    dept_total_orders=('reordered', 'count'),
    dept_reorder_rate=('reordered', 'mean')
).reset_index()

dept_reorders = prior_data[prior_data['reordered'] == 1].groupby('department')['reordered'].count()
dept_total = prior_data.groupby('department')['reordered'].count()
dept_reorder_share = (dept_reorders / dept_total).fillna(0).rename('dept_reorder_share').reset_index()

# Merge with Product Info
product_info = prior_data[['product_id', 'aisle', 'department']].drop_duplicates()

product_features = product_features.merge(product_info, on='product_id', how='left')
product_features = product_features.merge(aisle_features, on='aisle', how='left')
product_features = product_features.merge(dept_features, on='department', how='left')
product_features = product_features.merge(dept_reorder_share, on='department', how='left')

# Final product-level feature table is ready
display(product_features)

,product_id,prod_total_purchases,prod_reorder_count,prod_reorder_rate,prod_avg_cart_position,prod_distinct_users,prod_first_cart_order_avg,prod_median_cart_position,aisle,department,aisle_total_orders,aisle_reorder_rate,aisle_distinct_products,dept_total_orders,dept_reorder_rate,dept_reorder_share
0,1,1852,1136,0.613391,5.801836,716,0.146868,4.0,cookies cakes,snacks,234065,0.548698,873,2887550,0.574180,0.574180
1,2,90,12,0.133333,9.888889,78,0.122222,8.0,spices seasonings,pantry,212092,0.152391,797,1875577,0.346721,0.346721
2,3,277,203,0.732852,6.415162,74,0.155235,4.0,tea,beverages,249341,0.527615,894,2690129,0.653460,0.653460
3,4,329,147,0.446809,9.507599,182,0.042553,8.0,frozen meals,frozen,390299,0.556655,880,2236432,0.541885,0.541885
4,5,15,9,0.600000,6.466667,6,0.000000,6.0,marinades meat preparation,pantry,62510,0.280627,409,1875577,0.346721,0.346721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49672,49684,9,1,0.111111,4.333333,8,0.000000,4.0,spirits,alcohol,28102,0.572344,195,153696,0.569924,0.569924
49673,49685,49,6,0.122449,9.571429,43,0.163265,6.0,frozen vegan vegetarian,frozen,99369,0.542171,189,2236432,0.541885,0.541885
49674,49686,120,84,0.700000,7.500000,36,0.091667,6.0,bread,bakery,584834,0.670168,557,1176787,0.628141,0.628141
49675,49687,13,6,0.461538,7.538462,7,0.230769,5.0,cat food care,pets,63421,0.620883,499,97724,0.601285,0.601285


In [14]:
product_features.to_pickle('product_features.pkl')

User X Product Features

In [16]:
# Filter needed columns
data = prior_data[['user_id', 'product_id', 'order_number', 'add_to_cart_order', 
                   'reordered', 'days_since_prior_order']]

# Compute user-product grouped stats
up = data.groupby(['user_id', 'product_id']).agg(
    up_order_count=('order_number', 'count'),
    up_first_order_number=('order_number', 'min'),
    up_last_order_number=('order_number', 'max'),
    up_avg_cart_position=('add_to_cart_order', 'mean'),
    up_total_reordered=('reordered', 'sum')
).reset_index()

# Flag if user ever reordered this product
up['up_reorder_flag'] = (up['up_total_reordered'] > 0).astype(int)

# Merge total user orders for calculating reorder ratio
user_total_orders = prior_data.groupby('user_id')['order_number'].max().rename('user_total_orders')
up = up.merge(user_total_orders, on='user_id', how='left')

# Reorder ratio and order rate since first
up['up_reorder_ratio'] = up['up_order_count'] / up['user_total_orders']
up['up_order_rate_since_first'] = up['up_order_count'] / (up['user_total_orders'] - up['up_first_order_number'] + 1)

# Orders since last purchase
up['up_orders_since_last'] = up['user_total_orders'] - up['up_last_order_number']

# Get days_since_prior_order per user order
user_order_days = prior_data.groupby(['user_id', 'order_number'])['days_since_prior_order'].first().reset_index()
user_order_days['days_since_prior_order'] = user_order_days['days_since_prior_order'].fillna(0)

# Cumulative days for each order
user_order_days['days_cumulative'] = user_order_days.groupby('user_id')['days_since_prior_order'].cumsum()

# Merge days to get days_since_last_order
last_order_days = user_order_days.rename(columns={
    'order_number': 'up_last_order_number',
    'days_cumulative': 'up_days_since_last_order'
})[['user_id', 'up_last_order_number', 'up_days_since_last_order']]

up = up.merge(last_order_days, on=['user_id', 'up_last_order_number'], how='left')

display(up.head())

,user_id,product_id,up_order_count,up_first_order_number,up_last_order_number,up_avg_cart_position,up_total_reordered,up_reorder_flag,user_total_orders,up_reorder_ratio,up_order_rate_since_first,up_orders_since_last,up_days_since_last_order
0,1,196,10,1,10,1.400000,9,1,10,1.0,1.000000,0,176.0
1,1,10258,9,2,10,3.333333,8,1,10,0.9,1.000000,0,176.0
2,1,10326,1,5,5,5.000000,0,0,10,0.1,0.166667,5,93.0
3,1,12427,10,1,10,3.300000,9,1,10,1.0,1.000000,0,176.0
4,1,13032,3,2,10,6.333333,2,1,10,0.3,0.333333,0,176.0


In [17]:
up.to_pickle('up.pkl')

In [18]:
train_orders = orders.merge(order_products_train, on = 'order_id', how = 'inner')
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [19]:
train_orders.drop(['eval_set', 'add_to_cart_order', 'order_id'], axis = 1, inplace = True)

In [20]:
train_orders.head()

,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered
0,1,11,4,8,14.0,196,1
1,1,11,4,8,14.0,25133,1
2,1,11,4,8,14.0,38928,1
3,1,11,4,8,14.0,26405,1
4,1,11,4,8,14.0,39657,1


In [21]:
train_users = train_orders.user_id.unique()
train_users[:10]

array([ 1,  2,  5,  7,  8,  9, 10, 13, 14, 17])

In [22]:
df = up[up.user_id.isin(train_users)]
df.head()

,user_id,product_id,up_order_count,up_first_order_number,up_last_order_number,up_avg_cart_position,up_total_reordered,up_reorder_flag,user_total_orders,up_reorder_ratio,up_order_rate_since_first,up_orders_since_last,up_days_since_last_order
0,1,196,10,1,10,1.400000,9,1,10,1.0,1.000000,0,176.0
1,1,10258,9,2,10,3.333333,8,1,10,0.9,1.000000,0,176.0
2,1,10326,1,5,5,5.000000,0,0,10,0.1,0.166667,5,93.0
3,1,12427,10,1,10,3.300000,9,1,10,1.0,1.000000,0,176.0
4,1,13032,3,2,10,6.333333,2,1,10,0.3,0.333333,0,176.0


In [23]:
df = df.merge(train_orders, on = ['user_id', 'product_id'], how = 'outer')
df.head()

,user_id,product_id,up_order_count,up_first_order_number,up_last_order_number,up_avg_cart_position,up_total_reordered,up_reorder_flag,user_total_orders,up_reorder_ratio,up_order_rate_since_first,up_orders_since_last,up_days_since_last_order,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,10.0,1.0,10.0,1.400000,9.0,1.0,10.0,1.0,1.000000,0.0,176.0,11.0,4.0,8.0,14.0,1.0
1,1,10258,9.0,2.0,10.0,3.333333,8.0,1.0,10.0,0.9,1.000000,0.0,176.0,11.0,4.0,8.0,14.0,1.0
2,1,10326,1.0,5.0,5.0,5.000000,0.0,0.0,10.0,0.1,0.166667,5.0,93.0,NaN,NaN,NaN,NaN,NaN
3,1,12427,10.0,1.0,10.0,3.300000,9.0,1.0,10.0,1.0,1.000000,0.0,176.0,NaN,NaN,NaN,NaN,NaN
4,1,13032,3.0,2.0,10.0,6.333333,2.0,1.0,10.0,0.3,0.333333,0.0,176.0,11.0,4.0,8.0,14.0,1.0


In [24]:
df['order_number'] = df['order_number'].fillna(df.groupby('user_id')['order_number'].transform('mean'))
df['order_dow'] = df['order_dow'].fillna(df.groupby('user_id')['order_dow'].transform('mean'))
df['order_hour_of_day'] = df['order_hour_of_day'].fillna(df.groupby('user_id')['order_hour_of_day'].transform('mean'))
df['days_since_prior_order'] = df['days_since_prior_order'].fillna(
    df.groupby('user_id')['days_since_prior_order'].transform('mean')
)

In [25]:
df.reordered.value_counts()

reordered
1.0    828824
0.0    555793
Name: count, dtype: int64

In [26]:
df.reordered.isnull().sum()

7645837

In [27]:
df = df[df.reordered != 0]

In [28]:
df.shape

(8474661, 18)

In [29]:
df.reordered.fillna(0, inplace = True)

df.isnull().sum()

/var/folders/d9/7q11w0fj4h37hzlh3m8w5rx80000gn/T/ipykernel_37602/3168525350.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.reordered.fillna(0, inplace = True)


user_id                      0
product_id                   0
up_order_count               0
up_first_order_number        0
up_last_order_number         0
up_avg_cart_position         0
up_total_reordered           0
up_reorder_flag              0
user_total_orders            0
up_reorder_ratio             0
up_order_rate_since_first    0
up_orders_since_last         0
up_days_since_last_order     0
order_number                 0
order_dow                    0
order_hour_of_day            0
days_since_prior_order       0
reordered                    0
dtype: int64

In [30]:
df.head()

,user_id,product_id,up_order_count,up_first_order_number,up_last_order_number,up_avg_cart_position,up_total_reordered,up_reorder_flag,user_total_orders,up_reorder_ratio,up_order_rate_since_first,up_orders_since_last,up_days_since_last_order,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,10.0,1.0,10.0,1.400000,9.0,1.0,10.0,1.0,1.000000,0.0,176.0,11.0,4.0,8.0,14.0,1.0
1,1,10258,9.0,2.0,10.0,3.333333,8.0,1.0,10.0,0.9,1.000000,0.0,176.0,11.0,4.0,8.0,14.0,1.0
2,1,10326,1.0,5.0,5.0,5.000000,0.0,0.0,10.0,0.1,0.166667,5.0,93.0,11.0,4.0,8.0,14.0,0.0
3,1,12427,10.0,1.0,10.0,3.300000,9.0,1.0,10.0,1.0,1.000000,0.0,176.0,11.0,4.0,8.0,14.0,0.0
4,1,13032,3.0,2.0,10.0,6.333333,2.0,1.0,10.0,0.3,0.333333,0.0,176.0,11.0,4.0,8.0,14.0,1.0


In [31]:
df = df.merge(product_features, on = 'product_id', how = 'left')
df = df.merge(user_features, on = 'user_id', how = 'left')
df.head()

,user_id,product_id,up_order_count,up_first_order_number,up_last_order_number,up_avg_cart_position,up_total_reordered,up_reorder_flag,user_total_orders_x,up_reorder_ratio,...,user_most_common_dow,user_reorder_ratio,user_min_order_gap,user_max_order_gap,user_avg_order_gap,user_median_days_between_orders,user_weekend_order_ratio,user_most_common_order_hour,user_hour_std,user_hour_entropy
0,1,196,10.0,1.0,10.0,1.400000,9.0,1.0,10.0,1.0,...,4,0.759259,0.0,30.0,20.259259,20.5,0.0,7,3.500355,2.659844
1,1,10258,9.0,2.0,10.0,3.333333,8.0,1.0,10.0,0.9,...,4,0.759259,0.0,30.0,20.259259,20.5,0.0,7,3.500355,2.659844
2,1,10326,1.0,5.0,5.0,5.000000,0.0,0.0,10.0,0.1,...,4,0.759259,0.0,30.0,20.259259,20.5,0.0,7,3.500355,2.659844
3,1,12427,10.0,1.0,10.0,3.300000,9.0,1.0,10.0,1.0,...,4,0.759259,0.0,30.0,20.259259,20.5,0.0,7,3.500355,2.659844
4,1,13032,3.0,2.0,10.0,6.333333,2.0,1.0,10.0,0.3,...,4,0.759259,0.0,30.0,20.259259,20.5,0.0,7,3.500355,2.659844


In [32]:
df.isnull().sum().sort_values(ascending = False)

user_id                            0
product_id                         0
up_order_count                     0
up_first_order_number              0
up_last_order_number               0
up_avg_cart_position               0
up_total_reordered                 0
up_reorder_flag                    0
user_total_orders_x                0
up_reorder_ratio                   0
up_order_rate_since_first          0
up_orders_since_last               0
up_days_since_last_order           0
order_number                       0
order_dow                          0
order_hour_of_day                  0
days_since_prior_order             0
reordered                          0
prod_total_purchases               0
prod_reorder_count                 0
prod_reorder_rate                  0
prod_avg_cart_position             0
prod_distinct_users                0
prod_first_cart_order_avg          0
prod_median_cart_position          0
aisle                              0
department                         0
a

In [33]:
df.columns

Index(['user_id', 'product_id', 'up_order_count', 'up_first_order_number',
       'up_last_order_number', 'up_avg_cart_position', 'up_total_reordered',
       'up_reorder_flag', 'user_total_orders_x', 'up_reorder_ratio',
       'up_order_rate_since_first', 'up_orders_since_last',
       'up_days_since_last_order', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'reordered',
       'prod_total_purchases', 'prod_reorder_count', 'prod_reorder_rate',
       'prod_avg_cart_position', 'prod_distinct_users',
       'prod_first_cart_order_avg', 'prod_median_cart_position', 'aisle',
       'department', 'aisle_total_orders', 'aisle_reorder_rate',
       'aisle_distinct_products', 'dept_total_orders', 'dept_reorder_rate',
       'dept_reorder_share', 'user_total_orders_y', 'user_total_products',
       'user_distinct_products', 'user_avg_basket_size',
       'user_avg_days_between_orders', 'user_avg_order_hour',
       'user_most_common_dow', 'user_reorder_rati

In [34]:
df.to_pickle('df.pkl')